In [37]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import os
import numpy as np

In [38]:

# Update this path to the location of your FER2013 dataset
base_dir = '../../Dataset/FER2013/' 
sub_dirs = ['train','test']

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

In [39]:
def extract_landmarks(results):
    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]
        
        landmarks = [(landmark.x, landmark.y, landmark.z) for landmark in face_landmarks.landmark]
        return landmarks
    else:
        return None

In [40]:
landmark_data = []

for sub_dir in sub_dirs:
    path = os.path.join(base_dir, sub_dir)
    for emotion in os.listdir(path):
        emotion_path = os.path.join(path, emotion)
        for image_file in os.listdir(emotion_path):
            image_path = os.path.join(emotion_path, image_file)

            # Load the image
            image = cv2.imread(image_path)
            if image is None:
                continue  # Skip if the image can't be loaded

            # Convert the image to RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Detect landmarks
            results = face_mesh.process(image_rgb)
            landmarks = extract_landmarks(results)

            if landmarks is not None:
                # Process landmarks as needed (e.g., normalization, flattening)
                processed_landmarks = np.array(landmarks).flatten()

                # Save the processed landmarks with the corresponding label
                label = emotion  # Or convert emotion to a numerical label if needed
                landmark_data.append((processed_landmarks, label))

In [45]:
import pandas as pd

data_for_df = []
for features, label in landmark_data:
    # Convert array of features to a list and append the label
    feature_list = features.tolist()
    feature_list.append(label)
    data_for_df.append(feature_list)

# Create DataFrame
columns = [f'feature_{i}' for i in range(len(data_for_df[0]) - 1)] + ['label']
landmark_df = pd.DataFrame(data_for_df, columns=columns)

# Save to CSV
landmark_df.to_csv('landmark_data.csv', index=False)